In [2]:
import argparse
import random
import csv
import pandas as pd
import numpy as np
!pip3 install treetaggerwrapper
import treetaggerwrapper
!pip3 install keras
from keras.models import load_model
from gensim.models import fasttext
from joblib import load
from sklearn.decomposition import PCA

/Users/lmoncla/.pyenv/versions/3.7.3/lib/python3.7/site-packages/treetaggerwrapper.py:740: FutureWarning: Possible nested set at position 8
  re.IGNORECASE | re.VERBOSE)
/Users/lmoncla/.pyenv/versions/3.7.3/lib/python3.7/site-packages/treetaggerwrapper.py:2044: FutureWarning: Possible nested set at position 152
  re.VERBOSE | re.IGNORECASE)
/Users/lmoncla/.pyenv/versions/3.7.3/lib/python3.7/site-packages/treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/Users/lmoncla/.pyenv/versions/3.7.3/lib/python3.7/site-packages/treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [3]:
def sentences_to_ngrams(sentences, ngram_size, fr_nouns_file):

    ngrams = []
    context_size = int(ngram_size / 2)
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr', TAGINENC='utf-8', TAGOUTENC='utf-8')

    with open(fr_nouns_file, "r") as file:
        fr_nouns = file.readlines()

    for s in sentences:
        s = s.replace(';', '')
        s = s.replace("'", chr(39))
        s = s.replace('\'', chr(39))
        s = s.replace("d\'", " deeee ")
        s = s.replace("l\'", " leeee ")

        sentence_tagged = treetaggerwrapper.make_tags(tagger.tag_text(s))
        sentence = list(np.array(sentence_tagged)[:, 0])  # getting only the token (not lemmas and POS)

        for i, token in enumerate(sentence):
            if token == "leeee":
                sentence[i] = "l\'"
            if token == 'deeee':
                sentence[i] = "d\'"

        index_left = sentence.index('[')
        index_right = sentence.index(']')

        phrase_ngram = []

        # add left context
        for i in range(context_size):
            try:
                phrase_ngram.append(sentence[index_left - context_size + i])
            except IndexError:
                # when there is not enough words (ex: pivot word starting the sentence)
                phrase_ngram.append(random.choice(fr_nouns).rstrip())

        # add pivot token(s) (can contain several tokens)
        phrase_ngram.append(' '.join(sentence[index_left + 1:index_right]))

        # add right context
        for i in range(context_size):
            try:
                phrase_ngram.append(sentence[index_right + 1 + i])
            except IndexError:
                # when there is not enough words (ex: pivot word starting the sentence)
                phrase_ngram.append(random.choice(fr_nouns).rstrip())

        ngrams.append(phrase_ngram)

    return ngrams

In [4]:
def vectorization(ngram_size, input_data, we_vector_size, fasttext_wv):

    data_vec = np.array([])

    for phrase in input_data:
        phrase_vec = np.array([])

        for word in phrase:
            word = word.replace("’", "\'")
            vec = fasttext_wv[word]
            phrase_vec = np.append(phrase_vec, vec)

        data_vec = np.append(data_vec, phrase_vec)

    data_vec = np.reshape(data_vec, (len(input_data), ngram_size, we_vector_size))

    return data_vec

In [8]:
input_data = './corpus/corpus_validation_mix.csv'
we_vector_size = 300
fr_nouns_file = './corpus/listedesnomfrancais.txt'
model_fasttext = './corpus/cc.fr.300.bin'

keras_models = ['GRU', 'MLP_PCA', 'MLP_AE']

np.random.seed(1)


In [9]:
print('** Load input data... \n')
df = pd.read_csv(input_data, delimiter=';', names=['idf', 'labels', 'sentences', 'pivot_words', 'src', 'alea'])

print(df.head(5))

y_test = df['labels']

** Load input data... 

   idf  labels                                          sentences  \
0  166       1  la balade peut se poursuivre autour du lac ou ...   
1  303       1  le sentier grimpe au-dessus du hameau avec un ...   
2  199       1  ( 9 ) poursuivre la descente vers la droite en...   
3  394       1  continuer un petit peu sur l'arête puis descen...   
4  313       1  (3) À la [patte d'oie], laisser le départ à dr...   

         pivot_words                src               alea  
0               cols  corpus_validation  0,085618299510875  
1            passage  corpus_validation  0,295723408093251  
2             église  corpus_validation  0,942577511847241  
3  sentier de montée  corpus_validation  0,543919977290256  
4        patte d'oie  corpus_validation    0,8779708338724  


In [10]:
print("** Loading fastText model...\n")
fasttext_model = fasttext.load_facebook_vectors(model_fasttext)

** Loading fastText model...



In [12]:
def preprocess_input(sentences, ngram_size, fr_nouns_file, fasttext_model, we_vector_size):
    print('** Transform sentences to ' + str(ngram_size) + ' ngrams... \n')
    ngrams_list = sentences_to_ngrams(sentences, ngram_size, fr_nouns_file)
    print(ngrams_list)

    print('** Vectorisation of inputs... \n')
    x_test = vectorization(ngram_size, ngrams_list, we_vector_size, fasttext_model)
    
    return x_test

In [31]:
def loadmodel(model_path, algorithm, keras_models):
    print('** Loading model ' + model_path + ' \n')

    if algorithm in keras_models:
        clf = load_model(model_path)
    else:
        clf = load(model_path)
        
    return clf


def prediction(clf, x_test, y_test, ngram_size, we_vector_size, algorithm):
    print('** Predicting... \n')

    if algorithm == 'RF' or algorithm == 'SVM' or algorithm == 'MLP_AE':
        x_test = np.reshape(x_test, (len(x_test), ngram_size * we_vector_size))

    if algorithm == 'MLP_PCA':
        x_test = np.reshape(x_test, (368, ngram_size * we_vector_size))
        pca = PCA(0.99)
        #x_test = pca.fit(x_test)
        x_test = pca.fit_transform(x_test)
        

    if algorithm in keras_models:
        score = clf.evaluate(x_test, y_test)
        acc = score[1]

    if algorithm == 'RF' or algorithm == 'SVM':
        acc = clf.score(x_test, y_test)

    return acc
    #print('Test accuracy:', acc)

In [16]:
model_path = './models/GRU_1gram.h5'
algorithm = 'GRU'
ngram_size = 1


x_test = preprocess_input(df['sentences'], ngram_size, fr_nouns_file, fasttext_model, we_vector_size)
model = loadmodel(model_path, algorithm, keras_models)

acc = prediction(model, x_test, y_test, ngram_size, we_vector_size, algorithm)
print('Validation accuracy:', acc)

** Transform sentences to 1 ngrams... 

[['cols'], ['passage'], ['église'], ['sentier de montée'], ["patte d' oie"], ['maisons'], ['snack-bar'], ['cols'], ['rochers'], ['chemin'], ['sentier'], ['parc'], ['gare de bifurcation'], ['sentier'], ['gare de bifurcation'], ['rive'], ['cabane'], ['lacets'], ['couloir'], ['bifurcation'], ['cairns'], ['sous-bois'], ['ruelle'], ['barre'], ["patte d' oie"], ['boulevard'], ['bambouseraie'], ['passage'], ['chemin'], ['passerelle'], ['col'], ['ruisseau'], ['brèches'], ['cirque'], ['couloir'], ['bornes'], ['chemin'], ['passerelle'], ['chemin'], ['carrefour'], ['lacets'], ['torrent'], ['téléphérique'], ['torrent'], ['circuit'], ['parking'], ['gare de bifurcation'], ['réservoir'], ['bifurcation'], ['téléphérique'], ['circuits'], ['couloir'], ['balises'], ['cascades'], ['rond-point'], ['tracé'], ['site'], ['chapelles'], ['bois'], ['pas'], ['chemins'], ['pente'], ['chalets'], ['chemin'], ['circuits'], ['chemin'], ['lignes'], ["patte d' oie"], ['abri'], ['p

In [17]:
model_path = './models/GRU_5grams.h5'
algorithm = 'GRU'
ngram_size = 5

x_test = preprocess_input(df['sentences'], ngram_size, fr_nouns_file, fasttext_model, we_vector_size)
model = loadmodel(model_path, algorithm, keras_models)

acc = prediction(model, x_test, y_test, ngram_size, we_vector_size, algorithm)
print('Validation accuracy:', acc)

** Transform sentences to 5 ngrams... 

[['vers', 'les', 'cols', 'voisins', 'mais'], ['avec', 'un', 'passage', 'dans', 'les'], ['atteindre', 'l’', 'église', 'en-dessous', 'du'], ['rejoindre', 'le', 'sentier de montée', 'un', 'peu'], ['À', 'la', "patte d' oie", ',', 'laisser'], ['milieu', 'des', 'maisons', ',', 'continuer'], ['.', 'Un', 'snack-bar', '(', 'souvent'], ['des', 'deux', 'cols', 'rencontrée', 'à'], ['sur', 'les', 'rochers', "d'", 'une'], ['le', 'même', 'chemin', ',', 'laisser'], ['laisser', 'le', 'sentier', 'qui', 'part'], ['sur', 'le', 'parc', ',', 'sauf'], ['.', 'Ancienne', 'gare de bifurcation', ',', 'elle'], ['prendre', 'le', 'sentier', 'à', 'gauche'], ['une', 'véritable', 'gare de bifurcation', 'des', 'branches'], [',', 'poursuivre', 'rive', 'droite', ','], [',', 'une', 'cabane', 'abandonnée', 'où'], ['porter', 'des', 'lacets', 'de', 'couleurs'], ['nommé', 'le', 'couloir', 'de', 'la'], ['à', 'une', 'bifurcation', '.', 'voyage'], ['repérer', 'des', 'cairns', 'qui', 'mènen

** Loading model ./models/GRU_5grams.h5 

** Predicting... 

7/7 [==============================] - 0s 2ms/step - loss: 0.7165 - accuracy: 0.7629
Validation accuracy: 0.7628865838050842


In [19]:
model_path = './models/GRU_7grams.h5'
algorithm = 'GRU'
ngram_size = 7

x_test = preprocess_input(df['sentences'], ngram_size, fr_nouns_file, fasttext_model, we_vector_size)
model = loadmodel(model_path, algorithm, keras_models)

acc = prediction(model, x_test, y_test, ngram_size, we_vector_size, algorithm)
print('Validation accuracy:', acc)

** Transform sentences to 7 ngrams... 

[['ou', 'vers', 'les', 'cols', 'voisins', 'mais', 'je'], ['hameau', 'avec', 'un', 'passage', 'dans', 'les', 'rochers'], ['à', 'atteindre', 'l’', 'église', 'en-dessous', 'du', 'hameau'], ['par', 'rejoindre', 'le', 'sentier de montée', 'un', 'peu', 'au'], [')', 'À', 'la', "patte d' oie", ',', 'laisser', 'le'], ['au', 'milieu', 'des', 'maisons', ',', 'continuer', 'par'], ['Canada', '.', 'Un', 'snack-bar', '(', 'souvent', 'abrégé'], ['intersection', 'des', 'deux', 'cols', 'rencontrée', 'à', 'la'], ['cassent', 'sur', 'les', 'rochers', "d'", 'une', 'netteté'], ['sur', 'le', 'même', 'chemin', ',', 'laisser', 'partir'], [',', 'laisser', 'le', 'sentier', 'qui', 'part', 'sur'], ['fermée', 'sur', 'le', 'parc', ',', 'sauf', 'les'], ['Trie-Château', '.', 'Ancienne', 'gare de bifurcation', ',', 'elle', 'est'], ['puis', 'prendre', 'le', 'sentier', 'à', 'gauche', 'après'], ['dernière', 'une', 'véritable', 'gare de bifurcation', 'des', 'branches', 'Est'], ['passe

** Loading model ./models/GRU_7grams.h5 

** Predicting... 

7/7 [==============================] - 0s 2ms/step - loss: 0.4734 - accuracy: 0.7887
Validation accuracy: 0.7886598110198975


In [32]:
model_path = './models/MLP_PCA_5grams.h5'
algorithm = 'MLP_PCA'
ngram_size = 5

x_test = preprocess_input(df['sentences'], ngram_size, fr_nouns_file, fasttext_model, we_vector_size)
model = loadmodel(model_path, algorithm, keras_models)

acc = prediction(model, x_test, y_test, ngram_size, we_vector_size, algorithm)
print('Validation accuracy:', acc)

** Transform sentences to 5 ngrams... 

[['vers', 'les', 'cols', 'voisins', 'mais'], ['avec', 'un', 'passage', 'dans', 'les'], ['atteindre', 'l’', 'église', 'en-dessous', 'du'], ['rejoindre', 'le', 'sentier de montée', 'un', 'peu'], ['À', 'la', "patte d' oie", ',', 'laisser'], ['milieu', 'des', 'maisons', ',', 'continuer'], ['.', 'Un', 'snack-bar', '(', 'souvent'], ['des', 'deux', 'cols', 'rencontrée', 'à'], ['sur', 'les', 'rochers', "d'", 'une'], ['le', 'même', 'chemin', ',', 'laisser'], ['laisser', 'le', 'sentier', 'qui', 'part'], ['sur', 'le', 'parc', ',', 'sauf'], ['.', 'Ancienne', 'gare de bifurcation', ',', 'elle'], ['prendre', 'le', 'sentier', 'à', 'gauche'], ['une', 'véritable', 'gare de bifurcation', 'des', 'branches'], [',', 'poursuivre', 'rive', 'droite', ','], [',', 'une', 'cabane', 'abandonnée', 'où'], ['porter', 'des', 'lacets', 'de', 'couleurs'], ['nommé', 'le', 'couloir', 'de', 'la'], ['à', 'une', 'bifurcation', '.', 'livre'], ['repérer', 'des', 'cairns', 'qui', 'mènent

** Predicting... 



ValueError: cannot reshape array of size 291000 into shape (368,1500)

In [33]:
!pip3 install scikit-learn==0.22
model_path = './models/SVM_5grams.joblib'
algorithm = 'SVM'
ngram_size = 5

x_test = preprocess_input(df['sentences'], ngram_size, fr_nouns_file, fasttext_model, we_vector_size)
model = loadmodel(model_path, algorithm, keras_models)

acc = prediction(model, x_test, y_test, ngram_size, we_vector_size, algorithm)
print('Validation accuracy:', acc)

** Transform sentences to 5 ngrams... 

[['vers', 'les', 'cols', 'voisins', 'mais'], ['avec', 'un', 'passage', 'dans', 'les'], ['atteindre', 'l’', 'église', 'en-dessous', 'du'], ['rejoindre', 'le', 'sentier de montée', 'un', 'peu'], ['À', 'la', "patte d' oie", ',', 'laisser'], ['milieu', 'des', 'maisons', ',', 'continuer'], ['.', 'Un', 'snack-bar', '(', 'souvent'], ['des', 'deux', 'cols', 'rencontrée', 'à'], ['sur', 'les', 'rochers', "d'", 'une'], ['le', 'même', 'chemin', ',', 'laisser'], ['laisser', 'le', 'sentier', 'qui', 'part'], ['sur', 'le', 'parc', ',', 'sauf'], ['.', 'Ancienne', 'gare de bifurcation', ',', 'elle'], ['prendre', 'le', 'sentier', 'à', 'gauche'], ['une', 'véritable', 'gare de bifurcation', 'des', 'branches'], [',', 'poursuivre', 'rive', 'droite', ','], [',', 'une', 'cabane', 'abandonnée', 'où'], ['porter', 'des', 'lacets', 'de', 'couleurs'], ['nommé', 'le', 'couloir', 'de', 'la'], ['à', 'une', 'bifurcation', '.', 'propos'], ['repérer', 'des', 'cairns', 'qui', 'mènen

/Users/lmoncla/.pyenv/versions/3.7.3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  
/Users/lmoncla/.pyenv/versions/3.7.3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator SVC from version 0.20.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  more_tags = base_class._more_tags(self)


AttributeError: 'SVC' object has no attribute 'break_ties'